# Analyzing (RE)PPTIS simulations using an MSM approach
This notebook contains an example workflow that can be used for estimating the crossing probability and pathlengths of a (RE)PPTIS simulation.

## 1. Import the necessary functions

In [9]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

import matplotlib.pyplot as plt
from pprint import pprint    # to print the vars of the pathensemble object
import numpy as np
import os
import glob

# Reading
from tistools import read_inputfile, get_LMR_interfaces, read_pathensemble, get_weights
from tistools import set_tau_distrib, set_tau_first_hit_M_distrib, cross_dist_distr, pathlength_distr
from tistools import collect_tau, collect_tau1, collect_tau2, collect_taum
from tistools import ACCFLAGS, REJFLAGS

# REPPTIS analysis
from tistools import get_lmr_masks, get_generation_mask, get_flag_mask, select_with_masks
from tistools import unwrap_by_weight, running_avg_local_probs, get_local_probs, get_global_probs_from_dict, get_global_probs_from_local

# MSM functions
from tistools import construct_M
from tistools import global_pcross_msm
from tistools import mfpt_to_first_last_state, mfpt_to_absorbing_states, construct_tau_vector
from tistools import create_labels_states, print_vector, print_all_tau

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 2. Load the simulation data

In [10]:

# Set the working directory
indir = "/Users/an/Documents/0_mfpt/repptis1/"  

# indir = "/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/PyRETIS3/toytis/simulations/sim_istarwell0_2108"
# indir = "/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/PyRETIS3/toytis/simulations/sim_repptismazegap2708"
# indir = "/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/i_star/simulations/RETIS_flat_br_noswap"
# indir = "/run/user/1001/gvfs/smb-share:server=files.ugent.be,share=eliawils,user=eliawils/shares/tw06_biommeda_pyretis/04.2024_MSM_elias/simulations/flat_w-walls/brownian-gamma5/30k-cycles/REPPTIS"
# indir = "/run/user/1001/gvfs/smb-share:server=files.ugent.be,share=eliawils,user=eliawils/shares/tw06_biommeda_abl/paper-msm-short-term/simulations-and-analysis/trypsin-benzamidine/infrepptisanalysis/tistools-mfptanalysis/"
indir = "/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/"
# indir = "/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/1D-experiments/REPPTIS"

# zero_minus_one = True if lambda_-1 interface is set
# zero_minus_one = False if lambda_-1 interface is not set
zero_minus_one = False

inputfile = indir + "/repptis.rst"    # When using PyRETIS, the input file for REPPTIS simulations is a .rst file
# inputfile = indir + "/retis3.rst"

# Move to working directory
os.chdir(indir)
print(os.getcwd())

# Set the ensemble folders and print them
folders = glob.glob(indir + "/0[0-9][0-9]")
folders = sorted(folders)
print(folders)

/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs
['/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/000', '/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/001', '/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/002', '/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/003', '/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/004', '/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/005', '/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/006', '/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/007', '/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/

In [11]:
# Reading all input
#===================
interfaces, zero_left, timestep = read_inputfile(inputfile)
LMR_interfaces, LMR_strings = get_LMR_interfaces(interfaces, zero_left)
pathensembles = []
for i,fol in enumerate(folders):
    print("#"*80)
    print(fol)
    pe = read_pathensemble(fol+"/pathensemble.txt")
    pe.set_name(fol)
    pe.set_interfaces([LMR_interfaces[i], LMR_strings[i]])
    if i==0:
        pe.set_zero_minus_one(zero_minus_one)   # TODO this is never used
        pe.set_in_zero_minus(True)
    if i==1:
        pe.set_in_zero_plus(True)
    w, _ = get_weights(pe.flags, ACCFLAGS, REJFLAGS, verbose = False)
    pe.set_weights(w)
    print("pathensemble info: ")
    pprint(vars(pe))
    pathensembles.append(pe)

    
    # Read order parameters order.txt/order.npy into path ensemble object, or load from order.npy file.
    # Saving order parameter files allows to speed up this notebook.
    #### CHANGE HERE ####
    pe.set_orders(load=False, acc_only=True, save=True)        # for the 1st time you run this notebook for a certain simulation, this will store .npy files
    # pe.set_orders(load=True, acc_only=True, save=False)                  # for the next times, you can read npy files (save=True/False is not important)
    # pe.set_orders(load=False, acc_only=True, save=False)     # if saving doesn't work

################################################################################
/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/000
pathensemble info: 
{'cyclenumbers': array([   0,    1,    2, ..., 9865, 9866, 9867]),
 'flags': array(['ACC', 'REJ', 'ACC', ..., 'REJ', 'REJ', 'REJ'], dtype='<U3'),
 'generation': array(['sh', 'sh', 'sh', ..., 'sh', 'sh', 'sh'], dtype='<U2'),
 'has_zero_minus_one': False,
 'in_zero_minus': True,
 'in_zero_plus': False,
 'interfaces': [[1.0, 1.0, 1.0], ['l_[0]', 'l_[0]', 'l_[0]']],
 'lambmaxs': array([1.04186, 1.04186, 1.03438, ..., 1.04281, 1.04281, 1.04281]),
 'lambmins': array([0.94194, 0.94194, 0.94659, ..., 0.9862 , 0.9862 , 0.9862 ]),
 'lengths': array([ 7,  1, 11, ...,  1,  1,  1]),
 'lmrs': array(['RMR', 'RMR', 'RMR', ..., 'RMR', 'RMR', 'RMR'], dtype='<U3'),
 'name': '/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/000',
 'ncycle': 9868,
 'newpa

## 3. Regular (RE)PPTIS analysis using tistools

### Analyze the REPPTIS simulation.

In [13]:
# Analysis output is saved to the data dictionary.
data = {}
for i, pe in enumerate(pathensembles):
    print("doing pathensemble {}".format(i))
    if i == 0:
        data[i] = {}
        continue  #  [0-] is not used for Pcross calculations

    # Classify the paths according to their path type.
    pathtypes = ("LML", "LMR", "RML", "RMR")
    pathtype_cycles = {}
    for ptype in pathtypes:
        pathtype_cycles[ptype] = unwrap_by_weight(
                (pe.lmrs == ptype).astype(int), pe.weights)
    
    # Running average analysis: ["running"]
    data[i] = {}
    data[i]["running"] = {}
    data[i]["running"]["plocal"] = {}
    for (ptype, p_loc) in zip(pathtypes, 
                              running_avg_local_probs(pathtype_cycles, 
                                                      pe.weights, tr = False)):
        data[i]["running"]["plocal"][ptype] = p_loc

    # Analysis using all data: ["full"]
    plocfull = get_local_probs(pe, tr=False)
    data[i]["full"] = {}
    for ptype in pathtypes:
        data[i]["full"][ptype] = plocfull[ptype]

    # data[i] have now ["full"] and ["running"]

doing pathensemble 0
doing pathensemble 1
Weights of the different paths:
wRMR = 0
wRML = 863
wLMR = 661
wLML = 7875
Local crossing probabilities:
pRMR = 0.0
pRML = 1.0
pLMR = 0.07743673851921275
pLML = 0.9225632614807873
Local crossing probabilities:
p2R = 0.0703266304926056
p2L = 0.9296733695073944
doing pathensemble 2
Weights of the different paths:
wRMR = 1133
wRML = 202
wLMR = 195
wLML = 3208
Local crossing probabilities:
pRMR = 0.8486891385767791
pRML = 0.15131086142322098
pLMR = 0.05730238025271819
pLML = 0.9426976197472818
Local crossing probabilities:
p2R = 0.28028704094554663
p2L = 0.7197129590544533
doing pathensemble 3
Weights of the different paths:
wRMR = 255
wRML = 144
wLMR = 229
wLML = 5534
Local crossing probabilities:
pRMR = 0.6390977443609023
pRML = 0.3609022556390977
pLMR = 0.03973624848169356
pLML = 0.9602637515183065
Local crossing probabilities:
p2R = 0.07854592664719247
p2L = 0.9214540733528075
doing pathensemble 4
Weights of the different paths:
wRMR = 4453
wRM

### Generate pathlength distribution figures, as in PyRETIS reports.

In [14]:
for i, pe in enumerate(pathensembles):
    upe = pe.unify_pe()
    # Pathlength distribution
    data[i]["pathlengths"] = pathlength_distr(upe)  # these might be used later or not! TODO
        
#=======================================
# make figures
makefigs = True 
if makefigs:
    for i, pe in enumerate(pathensembles):     
        if i == 0:
            continue
        # Cross distances distribution
        L, M, R, lmlpercs, lmllambs, rmrpercs, rmrlambs = cross_dist_distr(pe)
        fig,ax = plt.subplots()
        ax.plot(lmllambs, lmlpercs, lw=1, c="g")
        ax.plot(rmrlambs, rmrpercs, lw=1, c="r")
        for lamb in (L,M,R):
            ax.axvline(lamb, color='k', linestyle='--', lw = 0.5)
        ax.set_xlabel('Cross distance')
        ax.set_ylabel('Frequency')
        ax.set_title("Ensemble {}. L = {}, M = {}, R = {}".format(
            pe.name, L, M, R))
        ax.set_ylim(0)
        fig.savefig(f"pathensemble_{i}_crossdist.pdf")
        plt.close(fig)

        # Pathlength distribution      
        for ptype in pathtypes:
            fig, ax = plt.subplots()
            ax.plot(data[i]["pathlengths"][ptype]["bin_centers"], 
                data[i]["pathlengths"][ptype]["hist"])
            ax.set_xlabel('Pathlength')
            ax.set_ylabel('Frequency')
            ax.set_title(f"{np.sum(data[i]['pathlengths'][ptype]['hist'])} " + \
                         f"{ptype} paths. ")
            ax.legend([f"mean = {data[i]['pathlengths'][ptype]['mean']:.2f}, " + \
                          f"std = {data[i]['pathlengths'][ptype]['std']:.2f}"])
            fig.savefig(f"pathensemble_{i}_pathlength_{ptype}.pdf")
            plt.close(fig)

Are all weights 1?  True
Are all paths accepted?  True
Are all weights 1?  True
Are all paths accepted?  True
Are all weights 1?  True
Are all paths accepted?  True
Are all weights 1?  True
Are all paths accepted?  True
Are all weights 1?  True
Are all paths accepted?  True
Are all weights 1?  True
Are all paths accepted?  True
Are all weights 1?  True
Are all paths accepted?  True
Are all weights 1?  True
Are all paths accepted?  True
Are all weights 1?  True
Are all paths accepted?  True
Are all weights 1?  True
Are all paths accepted?  True
Are all weights 1?  True
Are all paths accepted?  True
Are all weights 1?  True
Are all paths accepted?  True


### Compute Pcross using in-house functions

In [15]:
# Global crossing probabilities (no error analysis)  
psfull = []
for i in range(1, len(pathensembles)):   # do not use the 0- ensemble
    psfull.append({"LMR": data[i]["full"]["LMR"], 
               "RML": data[i]["full"]["RML"], 
               "RMR": data[i]["full"]["RMR"],
               "LML": data[i]["full"]["LML"]})

Pminfull, Pplusfull, Pcrossfull = get_global_probs_from_dict(psfull)

In [16]:
# Make a figure of the global crossing probabilities
fig, ax = plt.subplots()
ax.set_yscale("log")
ax.plot(Pcrossfull, "o", c = "r")
ax.errorbar([i for i in range(len(Pcrossfull))], Pcrossfull, fmt="-o", c = "b", ecolor="r", capsize=6)


ax.set_xlabel("intf")
ax.set_ylabel(r"$P_A(\lambda_i|\lambda_A)$")
ax.set_xticks(np.arange(len(interfaces)))
fig.tight_layout()
fig.show()
fig.savefig("Global_probs.pdf")

print("This should be the same as the repptis_report.pdf value:", Pcrossfull[-1])
print("which is the case!")
print(Pcrossfull)
print([Pcrossfull[i]/Pcrossfull[i-1] for i in range(1,len(Pcrossfull))])
print("Here, the load immediately disappeared. For a simulation where this is")
print("not the case, the above code should be adapted a little bit.")

This should be the same as the repptis_report.pdf value: 8.208294761348493e-05
which is the case!
[1.0, 0.07743673851921275, 0.004437309436158238, 0.0009529139181032057, 0.0005387254837279195, 0.0003052203223407093, 0.0001887025982107923, 0.00011817032864331779, 9.66955385532134e-05, 9.236781962183003e-05, 8.511537468626082e-05, 8.208294761348493e-05]
[0.07743673851921275, 0.05730238025271819, 0.21475038687593206, 0.5653453827185811, 0.5665600227942053, 0.6182504387769718, 0.6262252335885399, 0.8182725703088858, 0.9552438613390447, 0.9214829908807852, 0.9643727460056006]
Here, the load immediately disappeared. For a simulation where this is
not the case, the above code should be adapted a little bit.


In [17]:
# TODO DONT INCLUDE??

# Construct lists of the local probs

# Or we can use the get_global_probs_from_local function, using lists of the local probs
# These do not use the 0- ensemble
pmps = [data[i]["full"]["LMR"] for i in range(1,len(pathensembles))]
pmms = [data[i]["full"]["LML"] for i in range(1,len(pathensembles))]
ppps = [data[i]["full"]["RMR"] for i in range(1,len(pathensembles))]
ppms = [data[i]["full"]["RML"] for i in range(1,len(pathensembles))]
a,b,c = get_global_probs_from_local(pmps, pmms, ppps, ppms)
print("This should be the same as the repptis_report.pdf value:", c[-1])
print(c)
print([c[i]/c[i-1] for i in range(1,len(c))])

This should be the same as the repptis_report.pdf value: 8.208294761348493e-05
[1.0, 0.07743673851921275, 0.004437309436158238, 0.0009529139181032057, 0.0005387254837279195, 0.0003052203223407093, 0.0001887025982107923, 0.00011817032864331779, 9.66955385532134e-05, 9.236781962183003e-05, 8.511537468626082e-05, 8.208294761348493e-05]
[0.07743673851921275, 0.05730238025271819, 0.21475038687593206, 0.5653453827185811, 0.5665600227942053, 0.6182504387769718, 0.6262252335885399, 0.8182725703088858, 0.9552438613390447, 0.9214829908807852, 0.9643727460056006]


## 4. Analysis using the MSM

### Construct transition matrix M

In [18]:
print(interfaces)
N = len(interfaces)
NS = 4*N-5
print("N", N)
# print("len pmms", len(pmms)) # TODO INCLUDE?
print("NS", NS)

labels1, labels2 = create_labels_states(N)

[1.0, 2.0, 3.0, 4.0, 5.0, 6.5, 8.0, 10.0, 12.0, 14.0, 16.0, 19.0]
N 12
NS 43


In [19]:
print("mm", pmms)
print("mp", pmps)
print("pm", ppms)
print("pp", ppps)
print("sum", np.array(pmms)+np.array(pmps))
print("sum", np.array(ppms)+np.array(ppps))
if N > 3:  
    M = construct_M(pmms, pmps, ppms, ppps, N)
elif N == 3:
    M = construct_M_N3(pmms, pmps, ppms, ppps, N)
else:
    raise ValueError("The amount of interfaces needs to be 3 at least!")

mm [0.9225632614807873, 0.9426976197472818, 0.9602637515183065, 0.7226148409893993, 0.9628205128205128, 0.8324754328497894, 0.8802660753880266, 0.5693069306930693, 0.5789473684210527, 0.849772382397572, 0.5850991114149009]
mp [0.07743673851921275, 0.05730238025271819, 0.03973624848169356, 0.2773851590106007, 0.03717948717948718, 0.16752456715021058, 0.1197339246119734, 0.4306930693069307, 0.42105263157894735, 0.15022761760242792, 0.4149008885850991]
pm [1.0, 0.15131086142322098, 0.3609022556390977, 0.04911381592995943, 0.27601809954751133, 0.1770416904625928, 0.39568345323741005, 0.1067484662576687, 0.19485433219825957, 0.2835249042145594, 0.4713638236505974]
pp [0.0, 0.8486891385767791, 0.6390977443609023, 0.9508861840700406, 0.7239819004524887, 0.8229583095374072, 0.60431654676259, 0.8932515337423312, 0.8051456678017405, 0.7164750957854407, 0.5286361763494025]
sum [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
sum [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [20]:
# We can print the transition matrix M and check that all rows sum to 1.
print("M")
print("shape", M.shape)
print("sum prob in rows", np.sum(M,axis=1))
print(M)

M
shape (43, 43)
sum prob in rows [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[[0.         0.92256326 0.07743674 ... 0.         0.         0.        ]
 [1.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]
 [1.         0.         0.         ... 0.         0.         0.        ]]


### Look at this Markov model
*INCLUDE?*

In [21]:
#import numpy.linalg
vals, vecs = np.linalg.eig(M)
print(vals)
vals, vecs = np.linalg.eig(M.T)
print(vals)
pprint(M)

[ 1.00000000e+00+0.00000000e+00j  9.86209221e-01+0.00000000e+00j
  9.61027043e-01+0.00000000e+00j  9.12772458e-01+0.00000000e+00j
  9.00998553e-01+0.00000000e+00j  7.23611706e-01+0.00000000e+00j
  6.61476277e-01+0.00000000e+00j  6.49638183e-01+0.00000000e+00j
  5.72405859e-01+0.00000000e+00j  4.76693084e-01+0.00000000e+00j
  3.54410694e-13+2.69488025e-01j  3.54410694e-13-2.69488025e-01j
 -9.99926623e-01+0.00000000e+00j -9.86393349e-01+0.00000000e+00j
 -9.60831074e-01+0.00000000e+00j -9.13083716e-01+0.00000000e+00j
 -9.00768633e-01+0.00000000e+00j -4.76693105e-01+0.00000000e+00j
 -5.72404939e-01+0.00000000e+00j -7.23625117e-01+0.00000000e+00j
 -6.61431851e-01+0.00000000e+00j -6.49673976e-01+0.00000000e+00j
 -9.47593537e-09+0.00000000e+00j  9.47593583e-09+0.00000000e+00j
  2.86837427e-16+0.00000000e+00j  3.60987247e-17+1.87445339e-16j
  3.60987247e-17-1.87445339e-16j -1.14546913e-16+0.00000000e+00j
  1.36327513e-16+0.00000000e+00j -1.07536848e-17+4.65725668e-17j
 -1.07536848e-17-4.657256

In [22]:
print("what if chain propagates")
print("A[0,:]")
# check stationary behavior
A = M
for n in range(10):
    A = np.dot(A,M)
    #print(A)
    print(A[0,:])
    print(np.sum(A[0,:]))  # is 1 indeed

what if chain propagates
A[0,:]
[0.92256326 0.         0.         0.         0.07299943 0.00443731
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.        ]
1.0
[0.00000000e+00 8.51122971e-01 7.14402900e-02 7.29994291e-02
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 4.26098741e-03 1.76322030e-04 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e

### Pcross with MSM

In [23]:
# Inspect Z and Y vectors

z1, z2, y1, y2 = global_pcross_msm(M)
print("Z")
print_vector(z1, labels1)
print_vector(z2, labels2)
print("Y")
print_vector(y1, labels1)
print_vector(y2, labels2)
print("\nGlobal crossing probability: ", y1[0][0])

Z
state 0-     : 0
state B      : 1
state 0+- LML: 0.0
state 0+- LMR: 0.0010600000617681004
state 0+- RML: 0.0
state 1+- LML: 0.0
state 1+- LMR: 0.018498360052291517
state 1+- RML: 0.0
state 1+- RMR: 0.018498360052291517
state 2+- LML: 0.01569935725786239
state 2+- LMR: 0.08613889046439108
state 2+- RML: 0.015699357257862392
state 2+- RMR: 0.08613889046439109
state 3+- LML: 0.06071710404398976
state 3+- LMR: 0.15236507292263404
state 3+- RML: 0.06071710404398976
state 3+- RMR: 0.15236507292263404
state 4+- LML: 0.1478638914487737
state 4+- LMR: 0.26893015178018925
state 4+- RML: 0.14786389144877365
state 4+- RMR: 0.26893015178018925
state 5+- LML: 0.23551367268418766
state 5+- LMR: 0.43498578393601844
state 5+- RML: 0.23551367268418766
state 5+- RMR: 0.43498578393601844
state 6+- LML: 0.39967090415985196
state 6+- LMR: 0.6946155482163535
state 6+- RML: 0.39967090415985196
state 6+- RMR: 0.6946155482163535
state 7+- LML: 0.5779108329421982
state 7+- LMR: 0.8488804017396624
state 7+- RML

### Pathlength analysis

In [24]:
# Setting path ensemble properties
#==================================
for i,fol in enumerate(folders):
    print(i)
    print("Calculating path lengths.")
    set_tau_distrib(pathensembles[i])
    print("Done.")

    if True:
        print("Calculating first hitting lengths to middle interface")
        set_tau_first_hit_M_distrib(pathensembles[i])
        print("Done.")

0
Calculating path lengths.
Done.
Calculating first hitting lengths to middle interface
Done.
1
Calculating path lengths.
Done.
Calculating first hitting lengths to middle interface
Done.
2
Calculating path lengths.
Done.
Calculating first hitting lengths to middle interface
Done.
3
Calculating path lengths.
Done.
Calculating first hitting lengths to middle interface
Done.
4
Calculating path lengths.
Done.
Calculating first hitting lengths to middle interface
Done.
5
Calculating path lengths.
Done.
Calculating first hitting lengths to middle interface
Done.
6
Calculating path lengths.
Done.
Calculating first hitting lengths to middle interface
Done.
7
Calculating path lengths.
Done.
Calculating first hitting lengths to middle interface
Done.
8
Calculating path lengths.
Done.
Calculating first hitting lengths to middle interface
Done.
9
Calculating path lengths.
Done.
Calculating first hitting lengths to middle interface
Done.
10
Calculating path lengths.
Done.
Calculating first hitting

In [25]:
# Additional information
#==================================
# Average path lengths per ensemble for each path type
print(indir[-20:])
pathtypes = ("LML", "LMR", "RMR", "RML", "LM*", "*M*", "***", "RM*", "L**", "**R", "R**")

print("=" * 80)
print("AVERAGE PATH LENGTHS BY ENSEMBLE AND PATH TYPE")
print("=" * 80)

for i, pe in enumerate(pathensembles):
    print(f"\nEnsemble {i} ({pe.name}):")
    print("-" * 50)
    
    # Get accepted paths only
    accepted_mask = np.isin(pe.flags, ACCFLAGS)
    total_accepted_count = np.sum(accepted_mask)
    
    if total_accepted_count > 0:
        # Calculate weighted average for all accepted paths
        accepted_lengths = pe.lengths[accepted_mask]
        accepted_weights = pe.weights[accepted_mask]
        total_weighted_avg = np.average(accepted_lengths, weights=accepted_weights)
        
        print(f"  All accepted paths: {total_weighted_avg:8.2f} (n={total_accepted_count:4d}, weighted)")
        print("-" * 30)
    
    for ptype in pathtypes:
        mask = (pe.lmrs == ptype) & accepted_mask
        if np.any(mask):
            lengths = pe.lengths[mask]
            weights = pe.weights[mask]
            weighted_avg = np.average(lengths, weights=weights)
            count = np.sum(mask)
            print(f"  {ptype:4s}: {weighted_avg:8.2f} (n={count:4d}, weighted)")
        else:
            print(f"  {ptype:4s}: {0:8.2f} (n={0:4d}, weighted)")

s_data_sc10_12intfs/
AVERAGE PATH LENGTHS BY ENSEMBLE AND PATH TYPE

Ensemble 0 (/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/000):
--------------------------------------------------
  All accepted paths:    63.53 (n=1980, weighted)
------------------------------
  LML :     0.00 (n=   0, weighted)
  LMR :     0.00 (n=   0, weighted)
  RMR :    63.53 (n=1980, weighted)
  RML :     0.00 (n=   0, weighted)
  LM* :     0.00 (n=   0, weighted)
  *M* :     0.00 (n=   0, weighted)
  *** :     0.00 (n=   0, weighted)
  RM* :     0.00 (n=   0, weighted)
  L** :     0.00 (n=   0, weighted)
  **R :     0.00 (n=   0, weighted)
  R** :     0.00 (n=   0, weighted)

Ensemble 1 (/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/001):
--------------------------------------------------
  All accepted paths:    68.42 (n=1992, weighted)
------------------------------
  LML :    45.59 (n=1780, weighted

In [26]:
# Compute taus for pathlength analysis
tau_mm, tau_mp, tau_pm, tau_pp = collect_tau(pathensembles)
tau1_mm, tau1_mp, tau1_pm, tau1_pp = collect_tau1(pathensembles)
tau2_mm, tau2_mp, tau2_pm, tau2_pp = collect_tau2(pathensembles)
taum_mm, taum_mp, taum_pm, taum_pp = collect_taum(pathensembles)

Collect tau
ensemble 0 /mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/000
ensemble 1 /mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/001
ensemble 2 /mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/002
ensemble 3 /mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/003
ensemble 4 /mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/004
ensemble 5 /mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/005
ensemble 6 /mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/006
ensemble 7 /mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTIS/trypsin-benzos/pyretis_data_sc10_12intfs/007
ensemble 8 /mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/MSM-REPPTI

In [27]:
# Look at computed taus
print("tau")
print_all_tau(pathensembles, tau_mm, tau_mp, tau_pm, tau_pp)
print("\ntau1")
print_all_tau(pathensembles, tau1_mm, tau1_mp, tau1_pm, tau1_pp)
print("\ntaum")
print_all_tau(pathensembles, taum_mm, taum_mp, taum_pm, taum_pp)
print("\ntau2")
print_all_tau(pathensembles, tau2_mm, tau2_mp, tau2_pm, tau2_pp)

tau
Index Name            mm           mp           pm           pp
-----------------------------------------------------
0     000            nan          nan          nan         61.5
1     001           43.6        246.4        136.9          nan
2     002         1465.8       7364.0      10233.5       2930.8
3     003         1084.6       2607.4       3108.3       3462.9
4     004         3237.6       3543.8       2651.8       1320.9
5     005         1056.4       2328.7       1753.8       1770.2
6     006         2129.1       8307.4       7944.5       2522.6
7     007        11286.7       6838.7      29403.4      12307.2
8     008         5906.7       7133.5       6865.9       3807.9
9     009         2559.4       3484.2       3274.9       2947.8
10    010         3015.6       2095.5       2015.8       1592.1
11    011         1727.6       2156.8       2022.4       2228.1

tau1
Index Name            mm           mp           pm           pp
----------------------------------------

In [28]:
# TODO include prints?
tau  = construct_tau_vector(N, NS, tau_mm, tau_mp, tau_pm, tau_pp)
tau1 = construct_tau_vector(N, NS, tau1_mm, tau1_mp, tau1_pm, tau1_pp)
taum = construct_tau_vector(N, NS, taum_mm, taum_mp, taum_pm, taum_pp)
tau2 = construct_tau_vector(N, NS, tau2_mm, tau2_mp, tau2_pm, tau2_pp)
tau_m = tau-tau1-tau2  # yes, this is the same thing as taum

print("tau")
print(tau)
print("\n")
print("tau1")
print(tau1)
print("taum")
print(taum)
print("tau2")
print(tau2)

print("\n")
print("tau = tau1+taum+tau2 => difference is", np.sum((tau-tau1-taum-tau2)**2))

tau
[   61.53252939    43.58628571   246.3903177    136.94669757
  1465.77244389  7363.95897436 10233.50990099  2930.81906443
  1084.64040477  2607.40611354  3108.32638889  3462.94509804
  3237.6405868   3543.75796178  2651.82608696  1320.88030541
  1056.35229218  2328.74384236  1753.79234973  1770.22291667
  2129.09612142  8307.44134078  7944.45806452  2522.56696738
 11286.74685139  6838.73148148 29403.37575758 12307.23015873
  5906.68985507  7133.46360153  6865.89655172  3807.86813187
  2559.39304813  3484.18933824  3274.91067961  2947.7556391
  3015.58125     2095.50757576  2015.83783784  1592.11306341
  1727.57126168  2156.75041186     0.        ]


tau1
[   0.            0.            0.          136.94669757  179.49657107
  217.16410256  200.28712871  289.47661077  231.16841344  204.1441048
  150.32638889  523.11372549  206.14914425  199.63057325   83.68695652
  222.65888165  172.91173673  131.3546798   253.42076503  405.37291667
  445.65092749  516.67318436  346.31290323  383.52

## 5. Flux calculation

### Collect tau for [0+]

In [29]:
# Construct g and h vectors
g1, g2, h1, h2 = mfpt_to_first_last_state(M, np.nan_to_num(tau1), np.nan_to_num(tau_m), np.nan_to_num(tau2)) #, doprint=True)
print("G")
print_vector(g1, labels1)
print_vector(g2, labels2)
print("H")
print_vector(h1, labels1)
print_vector(h2, labels2)
print("\ntau [0+]: ", h1[0])

[ 0 42] g1:  [[0.]
 [0.]] D [[1. 0.]
 [0. 0.]
 [1. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 1.]] tp [[   43.58628571]
 [  246.3903177 ]
 [    0.        ]
 [ 1286.27587282]
 [ 7146.79487179]
 [10033.22277228]
 [ 2641.34245366]
 [  853.47199133]
 [ 2403.26200873]
 [ 2958.        ]
 [ 2939.83137255]
 [ 3031.49144254]
 [ 3344.12738854]
 [ 2568.13913043]
 [ 1098.22142376]
 [  883.44055545]
 [ 2197.38916256]
 [ 1500.3715847 ]
 [ 1364.85      ]
 [ 1683.44519393]
 [ 7790.76815642]
 [ 7598.14516129]
 [ 2139.04094379]
 [10644.45843829]
 [ 6084.53703704]
 [28162.29090909]
 [11218.31746032]
 [ 5030.85507246]
 [ 5487.43295019]
 [ 6494.52490421]
 [ 3271.06318681]
 [ 1998.68716578]
 [ 2864.05330882]
 [ 27

### The flux

In [30]:
flux = 1/(tau[0]+h1[0][0])
dt = 0.002 # Change if needed
sc = 10
flux /= (dt*sc)
# print(flux/(dt*sc), "1/time")
print(flux, "1/ps")

0.10647197407583793 1/ps


## 6. The rate constant
We can compute an accurate rate constant using only our MSM.

In [31]:
# rate constant = flux * Pcross

print("The rate constant k is: ", flux*y1[0][0], "1/ps")

The rate constant k is:  8.739533470371418e-06 1/ps


## 7. Direct rate computation via $\tau_{\mathcal{A},1}$ 

In [32]:
# Construct g and h vectors
absor = np.array([NS - 1])
kept = np.array([i for i in range(NS) if i not in absor])

g1, g2, h1, h2 = mfpt_to_absorbing_states(M, np.nan_to_num(tau1), np.nan_to_num(tau_m), np.nan_to_num(tau2), absor, kept, remove_initial_m=False, doprint=True)
print("G")
print_vector(g1, labels1[-1])
print_vector(g2, [labels1[0]] + labels2)
print("H")
print_vector(h1, labels1[-1])
print_vector(h2, [labels1[0]] + labels2)
print("interesting")
print(h2[0])
mfpt = h2[0][0]  # tau_A,1

[42] g1:  [[0.]] D [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]] tp [[   61.53252939]
 [   43.58628571]
 [  246.3903177 ]
 [    0.        ]
 [ 1286.27587282]
 [ 7146.79487179]
 [10033.22277228]
 [ 2641.34245366]
 [  853.47199133]
 [ 2403.26200873]
 [ 2958.        ]
 [ 2939.83137255]
 [ 3031.49144254]
 [ 3344.12738854]
 [ 2568.13913043]
 [ 1098.22142376]
 [  883.44055545]
 [ 2197.38916256]
 [ 1500.3715847 ]
 [ 1364.85      ]
 [ 1683.44519393]
 [ 7790.76815642]
 [ 7598.14516129]
 [ 2139.04094379]
 [10644.45843829]
 [ 6084.53703704]
 [28162.29090909]
 [11218.31746032]
 [ 5030.85507246]
 [ 5487.43295019]
 [ 6494.52490421]
 [ 3271.06318681]
 [ 1998.68716578]
 [ 2864.05330882]
 [ 2772.82135922]
 [ 2389.31203008]
 [ 2364.45580357]
 [ 1689.28787879]
 [ 1765.22393822]
 [ 1207.83040489]
 [ 13

In [33]:
k_flux_pcross = flux * y1[0][0]
k_mfpt = 1 / (mfpt*dt*sc)

print(f"Rate constant from P_cross × flux: {k_flux_pcross:.10e} [1/ps]")
print(f"Rate constant from MFPT:           {k_mfpt:.10e} [1/ps]")
print(f"Relative difference:               {abs(k_flux_pcross - k_mfpt)/k_mfpt*100:.2f}%")


Rate constant from P_cross × flux: 8.7395334704e-06 [1/ps]
Rate constant from MFPT:           8.7395334704e-06 [1/ps]
Relative difference:               0.00%
